In [1]:
# 1. limpieza de datos | data cleaning
# 1.1 libraries

    # manipulación de datos | data manipulation
import pandas as pd
import numpy as np

    # visual
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib_venn import venn2

    # análisis estadístico | statistical analysis
from scipy.stats import (
    ttest_ind,
    chi2_contingency,
    norm,
    mannwhitneyu,
    shapiro,
    probplot
)
from statsmodels.stats.proportion import (
    proportions_ztest,
    proportions_chisquare
)
import statsmodels.api as sm

    # configuración de pandas | pandas display settings
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
# 1.2 unir verticalmente y explorar los datasets de peatones | merge vertically and explore pedestrians datasets    
    # verificar columnas corrompidas antes de unir | columns that are mismatched or corrupted before you merge
    # load and fix each file | cargar archivos 
def load_and_fix(fname):
    df = pd.read_excel(f'../data/raw/{fname}')
    # fix corrupted column name | arreglar nombre de columna corrupto
    df.columns = [col.replace('Ãº', 'ú') for col in df.columns]
    # fill correct column if both exist | rellenar nombre de columna
    if 'Número_distrito' in df.columns and 'NÃºmero_distrito' in df.columns:
        df['Número_distrito'] = df['Número_distrito'].fillna(df['NÃºmero_distrito'])
        df.drop(columns=['NÃºmero_distrito'], inplace=True)
    # rename for consistency | renombrar para consistencia
    df.rename(columns={'Número_distrito': 'numero_distrito'}, inplace=True)
    return df

# apply to all files | aplicar a todos los archivos
df_peatones22 = load_and_fix('datos_madrid_aforo_peatones_2022.xlsx')
df_peatones23 = load_and_fix('datos_madrid_aforo_peatones_2023.xlsx')
df_peatones24 = load_and_fix('datos_madrid_aforo_peatones_2024.xlsx')

# merge | unir
df_pedestrians = pd.concat([df_peatones22, df_peatones23, df_peatones24], ignore_index=True)
# head
print("Head:\n", df_pedestrians.head(3))
# info
df_pedestrians.info()

Head:
                 fecha      hora    identificador  peatones        device_id  \
0 2022-09-01 00:00:00  00:00:00  PERM_PEA02_PM01    136.00  PERM_PEA02_PM01   
1 2022-09-01 01:00:00  01:00:00  PERM_PEA02_PM01    129.00  PERM_PEA02_PM01   
2 2022-09-01 02:00:00  02:00:00  PERM_PEA02_PM01    138.00  PERM_PEA02_PM01   

   numero_distrito distrito                          direccion  \
0             1.00   Centro  Calle Fuencarral 22, 28004 Madrid   
1             1.00   Centro  Calle Fuencarral 22, 28004 Madrid   
2             1.00   Centro  Calle Fuencarral 22, 28004 Madrid   

  observaciones_direccion  latitude  longitude  
0         Calle peatonal   40422009   -3700892  
1         Calle peatonal   40422009   -3700892  
2         Calle peatonal   40422009   -3700892  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371179 entries, 0 to 371178
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   ------------

In [ ]:


# renombrar columnas | rename columns
df_pedestrians.rename(
    columns={
        'fecha' : 'timestamp',
        'identificador': 'sensor_id', 
        'peatones': 'pedestrian_count',
        'numero_distrito ': 'district_id',
        'distrito' : 'district_name',
        'latitude' : 'lat',
        'longitude' : 'lon' }, 
        inplace=True)

Head:
             timestamp        sensor_id  pedestrian_count  numero_distrito  \
0 2022-09-01 00:00:00  PERM_PEA02_PM01            136.00             1.00   
1 2022-09-01 01:00:00  PERM_PEA02_PM01            129.00             1.00   
2 2022-09-01 02:00:00  PERM_PEA02_PM01            138.00             1.00   

  district_name       lat      lon  
0        Centro  40422009 -3700892  
1        Centro  40422009 -3700892  
2        Centro  40422009 -3700892  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371179 entries, 0 to 371178
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   timestamp         371179 non-null  datetime64[ns]
 1   sensor_id         371179 non-null  object        
 2   pedestrian_count  370853 non-null  float64       
 3   numero_distrito   312162 non-null  float64       
 4   district_name     312162 non-null  object        
 5   lat               371179 non-null  i

In [13]:
# check for nulls, uniques and duplicates in numerical and categorical columns | nulos, unicos y duplicados en columnas
df_pedestrians['timestamp'] = pd.to_datetime(df_pedestrians['timestamp'], errors='coerce'); print(f"\n.) Numerical Columns: {df_pedestrians[['timestamp'] + df_pedestrians.select_dtypes(include=['number']).columns.tolist()].columns.tolist()}\n\n.) Nulls for Numerical Columns:\n{df_pedestrians[['timestamp'] + df_pedestrians.select_dtypes(include=['number']).columns.tolist()].isnull().sum()}\n\n.) Duplicate rows for Numerical: {df_pedestrians[['timestamp'] + df_pedestrians.select_dtypes(include=['number']).columns.tolist()].duplicated().sum()} \n\n.) Uniques For Numerical Columns:\n{df_pedestrians[['timestamp'] + df_pedestrians.select_dtypes(include=['number']).columns.tolist()].nunique().sort_values(ascending=False)}\n\n.] Categorical Columns: {df_pedestrians.select_dtypes(include=['object']).columns.tolist()}\n\n.] Nulls for Categorical Columns:\n{df_pedestrians.select_dtypes(include=['object']).isnull().sum()}\n\n.] Duplicate rows for Categorical: {df_pedestrians.select_dtypes(include=['object']).duplicated().sum()}\n\n.] Uniques For Categorical Columns:\n{df_pedestrians.select_dtypes(include=['object']).nunique().sort_values(ascending=False)}")


.) Numerical Columns: ['timestamp', 'pedestrian_count', 'numero_distrito', 'lat', 'lon']

.) Nulls for Numerical Columns:
timestamp               0
pedestrian_count      326
numero_distrito     59017
lat                     0
lon                     0
dtype: int64

.) Duplicate rows for Numerical: 468 

.) Uniques For Numerical Columns:
timestamp           16054
pedestrian_count     8621
lat                    31
lon                    31
numero_distrito         6
dtype: int64

.] Categorical Columns: ['sensor_id', 'district_name']

.] Nulls for Categorical Columns:
sensor_id            0
district_name    59017
dtype: int64

.] Duplicate rows for Categorical: 371147

.] Uniques For Categorical Columns:
sensor_id        31
district_name     7
dtype: int64
